# 建立推薦程式(用CV推薦職缺)，使用訓練好的模型分類，計算相似度

程式步驟 (使用 scikit-learn, Word2Vec)

> 1. 使用者輸入CV，以及工作條件

> 2. 清洗CV，轉換成BOW， 丟入訓練模型，顯示預測分類結果。

> 3. 利用預測分類結果，與使用者輸入工作條件，從mongoDB撈取職缺

> 4. 將CV轉換成Word2Vec可用BOW

> 5. 計算CV與職缺的相似度，推薦相似度最近的10筆工作 (使用Word2Vec模型)


效果展示

> 測試CV內容

    '''最近工作\n\n公司：\tXX醫院\t行業：\t醫療/護理/衛生\n職位：\t醫生\n最高學歷\t\n
    最高學歷\n\n學校：\t中醫藥大學\n學歷：\t本科\t專業：\t\n醫藥學\n工作經驗\t\n工作經驗\n\n公司：\t\n
    XX醫院\n2012/7–2017/7\n職位：\t醫生\n行業：\t醫療/護理/衛生\n部門：\t醫藥部\n工作內容：\n1.
    了解各種儀器設備的使用方法。2.參與手術工作，鍛煉手術操作能力。3.熟悉實際操作中所出現的問題並通過各種
    方法避免和克服。4.勤學好問，大膽展示自我，學會了要禮貌待人，要踏實幹事，要提高個人綜合素質。\n教育經歷\t\n
    教育經歷\n\n學校：\t\n中醫藥大學\n2007/9–2011/6\n專業：\t醫藥學\t本科\t\n自我評價\t\n自我評價\n\n在校期
    間，學習了解剖學、生物化學、生理學、病理學、精神學等等課程，在校成績優異，擔任學生幹部的職務，曾多次獲得過
    學校獎學金，平時積極主動參加校內活動，曾負責在專業內組織過醫學演講。學習態度端正，能夠主動學習，認真對待每
    一次學習的機會，我希望在學習理論知識的同時，能夠增強自己的實踐經驗，我 相信沒有做不到，只有不想做。\n求職
    意向\t\n求職意向\n\n到崗時間：\t一個月之內\n工作性質：\t全職\n希望行業：\t醫療/護理/衛生\n目標地點：\t
    北京\n期望月薪：\t面議/月\n目標職能：\t醫生\n語言能力\t\n語言能力\n\n英語：\t\n良好\n\n聽說：\t\n良好
    \n\n讀寫：\t\n良好\n\n證書\t\n證書\n\n大學英語四級'''

> 推薦結果

    0. 超音波師/醫檢師(台北分部) 送子鳥診所
    --------------------
    1. [北]產後護理-哺育顧問/泌乳指導/泌乳諮詢(薪資優渥) 宣捷幹細胞生技股份有限公司
    --------------------
    2. 北區臨床專員 服杏企業有限公司
    --------------------
    3. 景美德威-牙醫助理 德威國際口腔醫療體系_德威生物科技股份有限公司
    --------------------
    4. 臨床牙體技術師牙醫助理 長島牙醫診所


## 1. Clean User input CV

模擬用戶輸入數據

In [1]:
# User input data

test_cv = '''最近工作\n\n公司：\tXX醫院\t行業：\t醫療/護理/衛生\n職位：\t醫生\n最高學歷\t\n
最高學歷\n\n學校：\t中醫藥大學\n學歷：\t本科\t專業：\t\n醫藥學\n工作經驗\t\n工作經驗\n\n公司：\t\n
XX醫院\n2012/7–2017/7\n職位：\t醫生\n行業：\t醫療/護理/衛生\n部門：\t醫藥部\n工作內容：\n1.
了解各種儀器設備的使用方法。2.參與手術工作，鍛煉手術操作能力。3.熟悉實際操作中所出現的問題並通過各種
方法避免和克服。4.勤學好問，大膽展示自我，學會了要禮貌待人，要踏實幹事，要提高個人綜合素質。\n教育經歷\t\n
教育經歷\n\n學校：\t\n中醫藥大學\n2007/9–2011/6\n專業：\t醫藥學\t本科\t\n自我評價\t\n自我評價\n\n在校期
間，學習了解剖學、生物化學、生理學、病理學、精神學等等課程，在校成績優異，擔任學生幹部的職務，曾多次獲得過
學校獎學金，平時積極主動參加校內活動，曾負責在專業內組織過醫學演講。學習態度端正，能夠主動學習，認真對待每
一次學習的機會，我希望在學習理論知識的同時，能夠增強自己的實踐經驗，我 相信沒有做不到，只有不想做。\n求職
意向\t\n求職意向\n\n到崗時間：\t一個月之內\n工作性質：\t全職\n希望行業：\t醫療/護理/衛生\n目標地點：\t
北京\n期望月薪：\t面議/月\n目標職能：\t醫生\n語言能力\t\n語言能力\n\n英語：\t\n良好\n\n聽說：\t\n良好
\n\n讀寫：\t\n良好\n\n證書\t\n證書\n\n大學英語四級'''

test_work_area = '台北市'
test_work_exp = 6
test_edu = '大學'


定義清洗函數

In [2]:
import jieba
import re

In [3]:
# data cleaning
def string_clean(CV_data): 
    '''
    input: string (original CV description)
    output: string (clean CV description)
    '''
    job_desc = CV_data.split('\n') # 根據換行符號轉乘 List格式
    job_words = ''
    
    for words in job_desc:
        words = words.replace('\t', ' ').replace('\r', ' ')
        words = re.sub(r'[^\w\s]',' ', words)  # remove all punctuations
        words = re.sub(r'\d+', ' ', words)  # remove all numbers
        words = words.strip() # remove white space
        words += ' '
        job_words += words
        
    return (job_words)

In [4]:
jieba.load_userdict('jieba_data/Jobcontent_dict.txt') # 指定辭典檔

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.734 seconds.
Prefix dict has been built successfully.


In [5]:
# 排除字元表單 stopword, 開啟 'Jobcontent_stopwords.txt'檔案

with open(file='jieba_data/Jobcontent_stopwords.txt',mode='r', encoding="UTF-8") as file:
    stop_words = file.read().split('\n')
    stop_words = [i.strip() for i in stop_words]

In [6]:
def isEnglish(s): # 檢查字元是否為英文
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def jieba_cut(data, stop_words):
    
    # 使用結巴斷詞
    seg_result = jieba.cut(data, cut_all=False)

    # 篩選斷詞，去掉單一中文字
    lst_seg = []
    
    for i in list(seg_result):
        i = i.strip()
        if len(i) < 1:  #排除空值
            continue
        elif i in stop_words: # 排除stopwords
            continue
        elif isEnglish(i) == False and len(i) == 1: #排除單一中文字
            continue
        else:
            lst_seg.append(i)
   
    return lst_seg

In [7]:
test_cv_clean = string_clean(test_cv)
test_cv_clean = jieba_cut(test_cv_clean, stop_words)
test_cv_clean = [' '.join(test_cv_clean)]

In [8]:
test_cv_clean

['XX 醫院 行業 醫療 護理 衛生 職位 醫生 學歷 學歷 學校 醫藥 大學 學歷 本科 專業 醫藥 XX 醫院 職位 醫生 行業 醫療 護理 衛生 部門 醫藥 儀器 設備 方法 手術 鍛煉 手術 操作能力 操作 通過 方法 避免 克服 勤學 好問 展示 幹事 教育 經歷 教育 經歷 學校 醫藥 大學 專業 醫藥 本科 評價 評價 解剖 生物 化學 病理 課程 成績 擔任 學生 幹部 職務 獲得過 學校 參加校 專業 組織 醫學 演講 對待 理論 知識 增強 到崗 全職 行業 醫療 護理 衛生 期望 醫生 語言 語言 英語 讀寫 證書 證書 大學 英語']

## 2. Transform data, and predict the classification

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from joblib import dump, load
import pickle

In [10]:
# The scikit-learn version is 0.22.1.

In [11]:
def cv_category_predict(cv_data):

    vectorizer = pickle.load(open("model/vectorizer.pickel", "rb")) 
    transform_content = vectorizer.transform(cv_data) # CountVectorizer transform
    X_test = transform_content.toarray()
    # vectorizer.inverse_transform(X_test)

    model = load('model/MultinomialNB.joblib') 
    y_pred = model.predict(X_test)  # MultinomialNB transform

    le = pickle.load(open("model/le.pickel", "rb"))
    y_pred_category = le.inverse_transform(y_pred)
    y_pred_category = y_pred_category[0]
    
    return y_pred_category

In [12]:
test_cv_category = cv_category_predict(test_cv_clean)

顯示預測結果

In [13]:
print('CV data after jieba: ', test_cv_clean) 
print('='*20)
print('Predicted category: ', test_cv_category)

CV data after jieba:  ['XX 醫院 行業 醫療 護理 衛生 職位 醫生 學歷 學歷 學校 醫藥 大學 學歷 本科 專業 醫藥 XX 醫院 職位 醫生 行業 醫療 護理 衛生 部門 醫藥 儀器 設備 方法 手術 鍛煉 手術 操作能力 操作 通過 方法 避免 克服 勤學 好問 展示 幹事 教育 經歷 教育 經歷 學校 醫藥 大學 專業 醫藥 本科 評價 評價 解剖 生物 化學 病理 課程 成績 擔任 學生 幹部 職務 獲得過 學校 參加校 專業 組織 醫學 演講 對待 理論 知識 增強 到崗 全職 行業 醫療 護理 衛生 期望 醫生 語言 語言 英語 讀寫 證書 證書 大學 英語']
Predicted category:  醫療╱保健服務類



## 3. 連線至職缺，利用條件篩選

#### 使用 mongoDB 連線篩選職缺

In [14]:
import pymongo
import pandas as pd
from secrets import HOST

In [15]:
def mongo_select_jobs(category, area = None,  workExp = None, edu = None): 
    
    global mycol
    client = pymongo.MongoClient(HOST) 

    db = client['final_104'] 
    mycol = db['final_data'] 

    
    filter_dict = {'jobCat_main': category} # create filter for mongoDB
    
    if area != None:  
        filter_dict['Work_area_clean'] = area # filter data by area

    data = mycol.find(filter_dict,{'_id':0}) # get data from mongoDB
    output = [i for i in data]
    
    # turn output into dataframe
    df = pd.DataFrame(output)
    
     # filter data by work experience
    if workExp is not None:
        lst_workExp = list(i for i in range(0, workExp+1))
        df = df[df['workExp_clean'].astype(int).isin(lst_workExp)]    
    
    # filter data by education status
    if edu is not None:
        df = df[df['Edu_clean'].str.contains("大學")]

    filter_result = df.values.tolist()

    # 將資料儲存成flask可用的dictionary格式

    dict_data = {} 

    for i in filter_result:
        jobURL, Job_Name, Company, Job_Description, concate_jieba = i[0], i[1], i[2], i[8], i[9]
        dict_data[jobURL] = { 
        'Job_Name': Job_Name,
        'Company': Company,
        'Job_Description': Job_Description,
        'concate_jieba': concate_jieba }
        
    return dict_data    

In [16]:
# 利用預測分類結果，與使用者輸入工作條件，從mongoDB撈取職缺
jobs_query = mongo_select_jobs(test_cv_category, test_work_area, test_work_exp, test_edu)
print('總共有幾筆資料: ' , len(jobs_query))

總共有幾筆資料:  921


## 4. 將結巴後的職缺轉換成BOW

In [17]:
 # 將結巴後的斷詞，利用CountVectorizer轉換減少字詞數量
def turn_content_BOW(content):
    X_content = [ " ".join(content)]
    vectorizer = pickle.load(open("model/vectorizer.pickel", "rb")) 
    transform_content = vectorizer.transform(X_content) 
    X_content_array = transform_content.toarray()
    return X_content_array

## 6. 計算相似度，使用 Word2Vec similarity

In [18]:
from gensim.models import Word2Vec

In [19]:
 # 將結巴後的斷詞，利用CountVectorizer轉換，確認字詞在模型內
def turn_content_BOW(content):
    X_content = [ " ".join(content)]
    vectorizer = pickle.load(open("model/vectorizer.pickel", "rb")) 
    transform_content = vectorizer.transform(X_content) 
    X_content = vectorizer.inverse_transform(transform_content)[0].tolist()
    return X_content

In [20]:
# 計算 Word2Vec similarity
def compute_similarity(cv_BOW, job_BOW, model_train):
    job_prob = model_train.wv.n_similarity(cv_BOW, job_BOW)    
    return job_prob

In [21]:
# 定義 recommendation function，顯示10筆結果
def show_recommendation_result(cv_clean, jobs_query, model_train):
    cv_BOW = turn_content_BOW(cv_clean)
    
    # 確認BOW內的字詞包含在模型內
    cv_BOW_for_Word2Vec = []
    for i in cv_BOW:
        if i in list(model_train.wv.vocab.keys()):
            cv_BOW_for_Word2Vec.append(i)
    
    cv_BOW = cv_BOW_for_Word2Vec
    
    # 將職缺資料轉成 Word2Vec格式
    lst_jobs_content = []
    lst_jobs_url = []

    for k,v in jobs_query.items():
        split_data = v['concate_jieba'][0].split(',')
        lst_jobs_content.append(split_data)
        lst_jobs_url.append(k)

    # 計算所有CV與職缺的similarity，排序後儲存10筆相關度最高職缺
    dict_prob_id = {}

    for i, j in enumerate(lst_jobs_content):
        job_url = lst_jobs_url[i]
        job_prob = compute_similarity(cv_BOW, j, model_train)
        dict_prob_id[job_prob] = job_url
        result = [(k, dict_prob_id[k]) for k in sorted(dict_prob_id.keys(), reverse = True)[0:10]]
    
    # 顯示結果為list
    list_ten_result = []

    for i in result:
        prob, url = i[0], i[1]
        job_dict = jobs_query[url]
        lst_result = [prob, job_dict['Job_Name'], job_dict['Company'], job_dict['Job_Description'], url]
        list_ten_result.append(lst_result)
        
    return list_ten_result

顯示預測結果

In [23]:
model_train = Word2Vec.load('model/Word2Vec.model') # 載入訓練好的Word2Vec模型
recommendation = show_recommendation_result(test_cv_clean, jobs_query, model_train) #計算推薦結果，顯示前10筆推薦

In [24]:
for i, j in enumerate(recommendation):
    print(i, j[1], j[2])
    print(j[3])
    print('-'*20)

0 超音波師/醫檢師(台北分部) 送子鳥診所
1. 人工生殖及試管嬰兒技術、藥物使用、婦科相關、檢查說明等生殖醫學諮詢 2. 超音波影像操作相關訓練及影像報告處理  3. 門診抽血4. 開刀房業務支援與訓練5. 公司活動 、專案策劃執行 ，培育第二專長與發展個人舞台6. 教育訓練、升遷制度完善，多項專業技能發展，培養多方位人才7. 免上大夜班、排班制度完善，週日固定休假、每月一次三天連休8. 工作環境一流、生活品質好、待遇好福利佳、享季獎金及績效獎金 9. 適合各醫學院醫技系，須有醫檢師執照 10.錄取後需於新竹總部受訓3~6個月，提供宿舍或租屋補助金
 
1.醫事檢驗師，或具醫用超音波婦產科技術師認證資格尤佳2.具備電腦操作能力、態度積極主動
--------------------
1 [北]產後護理-哺育顧問/泌乳指導/泌乳諮詢(薪資優渥) 宣捷幹細胞生技股份有限公司
【本職位負責宣捷集團關係企業之相關工作】1. 協助母乳哺育，針對哺乳相關問題提供熟練及專業的處理。 2. 提供專業的哺育資訊和協助。 3. 薪資結構:每月底薪NT$25,000~35,000(依學經歷及證照調整)+優渥獎金。 
 
1. 具產兒或泌乳相關知識、技巧及服務經驗尤佳。2. 完成泌乳專業培訓課程者尤佳。3. 抗壓性高，具備溝通協調及團隊合作能力。4. 工作地點為新北市、台北市。
--------------------
2 北區臨床專員 服杏企業有限公司
1. 協助新產品研究報告。2. 臨床案例記錄。3.客戶端之教育訓練與Demo 。需具備職能：英文聽說讀寫能力、物理治療師職能、文書處理能力、溝通協調能力。個性特質：細心獨立、認真負責、具有耐心及毅力、有團隊意識、樂於學習成長、互助合作與分享。
 
醫工畢業佳 物理治療師,護士佳
--------------------
3 景美德威-牙醫助理 德威國際口腔醫療體系_德威生物科技股份有限公司
1.擔心大學畢業沒有一技之長嗎?本院提供完整在職教育訓練，無經驗者可，在院可學得一技之長，提升自己職場能力。2.周日固定休診，遇國定假日休診3.協助醫師看診與跟診4.協助患者做好口腔衛生及保健5.清潔、消毒器械6.勤奮具上進心，有意願接受院所長期培訓成為優秀專業人員佳8.本院為全省最完整口腔醫療體系，擁有牙科各相關產業，有興趣者可朝多元發展**試用